In [14]:
import requests
import pandas as pd
# import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
# from alpaca_trade_api.rest import REST
# from finvizfinance.quote import finvizfinance

In [10]:
pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 1.9 MB/s  0:00:05 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 1.8 MB/s  0:00:02 eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [15]:
def fetch_crypto_history(symbol, twelve_key, days=100):
    symbol = symbol.replace('-', '/')  # Convert BTC-USD → BTC/USD for Twelve Data  
    url = "https://api.twelvedata.com/time_series"
    params = {
        "symbol": symbol,
        "interval": "1day",
        "outputsize": days,
        "apikey": twelve_key
    }
    try:
        r = requests.get(url, params=params).json()
        if "values" not in r:
            print(f"❌ No crypto data for {symbol}: {r}")
            return pd.DataFrame()
        df = pd.DataFrame(r["values"])
        df["datetime"] = pd.to_datetime(df["datetime"])
        df.set_index("datetime", inplace=True)
        df = df.sort_index()
        df["close"] = df["close"].astype(float)
        return df[["close"]]
    except Exception as e:
        print(f"❌ Crypto fetch error ({symbol}): {e}")
        return pd.DataFrame()

In [16]:
fetch_crypto_history('BTC-USD','fb22b25e80884e8b89ef865a3ddb9802',days=100)

,close
datetime,
2025-05-26,109464.32
2025-05-27,108978.46
2025-05-28,107819.29
2025-05-29,105572.58
2025-05-30,104033.26
...,...
2025-08-29,108378.32
2025-08-30,108827.93
2025-08-31,108247.95


In [70]:
APCA_API_KEY_ID = 'AKLE4VA1CEHBBHXNXCHZ'
APCA_API_SECRET_KEY = 'SQiQpXxbecEhS2p5z9l2olfhA135AuMktWUu12pm'
LOOKBACK_DAYS = 400

In [38]:
# Initialize Alpaca
alpaca = REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, base_url="https://api.alpaca.markets")

In [40]:
# API_KEY = 'fb22b25e80884e8b89ef865a3ddb9802'
# BASE_URL = 'https://api.twelvedata.com/price'
# LOOKBACK_DAYS = 600

In [42]:
# ---------- LOAD YOUR PORTFOLIO FROM CSV ----------
df = pd.read_csv('robinhood_portfolio_combined2.csv')

In [44]:
# Separate out stocks/ETFs and crypto holdings
stocks_df = df[df['Classification'].isin(['Stock', 'ETF'])]
crypto_df = df[df['Classification'] == 'Crypto']

In [46]:
df.head()

,Symbol,Classification,Quantity Owned
0,AAPL,Stock,5.010683
1,ABBV,Stock,0.148790
2,ADBE,Stock,0.074650
3,ALB,Stock,19.197652
4,AMD,Stock,0.206640


In [48]:
all_tickers_in_your_portfolio =  df[df['Classification']=='Stock']["Symbol"].dropna().unique().tolist()

In [50]:
# Input target allocation
target_allocation = {
    "stocks": 40,
    "crypto": 10,
    "bonds": 20,
    "currencies": 30
}

# Build the portfolio dict
portfolio = {
    "stocks": {row.Symbol: row._3 for row in stocks_df.itertuples()},  
    "crypto": {f"{row.Symbol}": row._3 for row in crypto_df.itertuples()},
    # Fixed‐value categories (UPDATE these if needed)
    "bonds":      15000,
    "currencies": 37960.68
}

In [52]:
from finvizfinance.quote import finvizfinance

tickers = list(all_tickers_in_your_portfolio)  # flatten stock/crypto etc.
fundamentals_dict = {}

for tk in tickers:
    try:
        stock = finvizfinance(tk)
        fundamentals_dict[tk] = stock.ticker_fundament()
    except:
        fundamentals_dict[tk] = {}  # fallback in case of error

In [53]:
def fetch_crypto_history(symbol, days=100):
    url = "https://api.twelvedata.com/time_series"
    params = {
        "symbol": symbol,
        "interval": "1day",
        "outputsize": days,
        "apikey": "fb22b25e80884e8b89ef865a3ddb9802"
    }
    try:
        r = requests.get(url, params=params).json()
        if "values" not in r:
            print(f"❌ No crypto data for {symbol}: {r}")
            return pd.DataFrame()
        df = pd.DataFrame(r["values"])
        df["datetime"] = pd.to_datetime(df["datetime"])
        df.set_index("datetime", inplace=True)
        df = df.sort_index()
        df["close"] = df["close"].astype(float)
        return df[["close"]]
    except Exception as e:
        print(f"❌ Crypto fetch error ({symbol}): {e}")
        return pd.DataFrame()

In [54]:
def fetch_stock_history(symbol, lookback_days=100):
    end = datetime.now()
    start = end - timedelta(days=lookback_days)

    try:
        # ✅ Use IEX feed to avoid SIP errors
        bars = alpaca.get_bars(
            symbol,
            timeframe="1Day",
            start=start.strftime('%Y-%m-%d'),
            end=end.strftime('%Y-%m-%d'),
            feed='iex'  # 👈 This is the key line
        ).df

        if bars.empty:
            print(f"⚠️ No data for {symbol} on IEX")
            return pd.DataFrame()

        bars = bars.tz_localize(None)
        bars = bars.rename(columns={"close": "close"})
        return bars[["close"]]

    except Exception as e:
        print(f"❌ Error fetching {symbol}: {e}")
        return pd.DataFrame()

In [55]:
# ---------- ADD INDICATORS (UPDATED) ----------
def add_indicators(df):
    # RSI
    delta = df['close'].diff()

    # 2) Separate gains and losses as Series (same index as df)
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    # 3) Rolling average of gains/losses
    roll_up   = gain.rolling(window=14, min_periods=14).mean()
    roll_down = loss.rolling(window=14, min_periods=14).mean()

    # 4) Relative Strength, then RSI
    rs = roll_up / roll_down
    df['RSI'] = 100 - (100 / (1 + rs))

    # 5) Other indicators (unchanged)
    df['MA20'] = df['close'].rolling(20).mean()
    df['MA50'] = df['close'].rolling(50).mean()
    
    # MACD (12, 26, 9)
    ema12 = df['close'].ewm(span=12, adjust=False).mean()
    ema26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    
    # Bollinger Bands (20-day, 2σ)
    ma20 = df['MA20']
    std20 = df['close'].rolling(20).std()
    df['BB_Upper'] = ma20 + 2*std20
    df['BB_Lower'] = ma20 - 2*std20
    
    # Volatility (20-day)
    df['Vol_20d'] = std20
    
    return df

In [56]:
def check_technical_signals(latest):
    rsi = latest["RSI"]
    macd = latest["MACD"]
    macd_sig = latest["MACD_Signal"]
    price = latest["close"]
    bb_upper = latest["BB_Upper"]
    bb_lower = latest["BB_Lower"]

    overbought = (rsi > 70) or (price > bb_upper)
    oversold = (rsi < 30) or (price < bb_lower)
    macd_bull = macd > macd_sig
    macd_bear = macd < macd_sig

    return {
        "overbought": overbought,
        "oversold": oversold,
        "macd_bear": macd_bear,
        "macd_bull": macd_bull
    }

In [57]:
def trend(ma20, ma50):
    if pd.isna(ma20) or pd.isna(ma50):
        return "N/A"
    return "↗️ Up" if ma20 > ma50 else "↘️ Down"

In [96]:
# Map score to decision
def score_to_action(score):
    if score >= 70:
        return "✅ Strong Buy"
    elif 40 <= score < 70:
        return "🔄 Optional Buy"
    elif -39 <= score <= 39:
        return "❌ Hold"
    elif -69 <= score < -40:
        return "🔄 Optional Sell"
    else:
        return "✅ Strong Sell"

In [98]:
# New intelligent scoring-based rebalance system

def intelligent_score(technicals, fundamentals, category_drift, alloc_pct, target_pct):
    score = 0
    reasons = []

    # --- Technicals ---
    if technicals.get("macd_bull"):
        score += 20
        reasons.append("MACD Bullish")
    elif technicals.get("macd_bear"):
        score -= 20
        reasons.append("MACD Bearish")

    if technicals.get("oversold"):
        score += 15
        reasons.append("RSI Oversold")
    elif technicals.get("overbought"):
        score -= 15
        reasons.append("RSI Overbought")

    # --- Allocation drift ---
    drift = alloc_pct - target_pct
    if drift < -3:
        score += 10
        reasons.append("Underweight")
    elif drift > 3:
        score -= 10
        reasons.append("Overweight")

    # --- Fundamentals ---
    try:
        recom = float(fundamentals.get("Recom", 0))
        target_price = float(fundamentals.get("Target Price", 0))
        current_price = float(fundamentals.get("Price", 0))
        peg = float(fundamentals.get("PEG", 0))
        pe = float(fundamentals.get("P/E", 0))
        beta = float(fundamentals.get("Beta", 1))
        short_float = float(fundamentals.get("Short Float", "0%" ).replace("%", ""))
        eps_growth = float(fundamentals.get("EPS next Y Percentage", "0%" ).replace("%", ""))
        sales_growth = float(fundamentals.get("Sales Q/Q", "0%" ).replace("%", ""))
    except:
        return 0, "⚠️ Data error"

    # Sentiment
    if 0 < recom < 2:
        score += 15
        reasons.append("Strong Analyst Rating")
    elif recom > 3:
        score -= 10
        reasons.append("Weak Analyst Rating")

    # Valuation
    if target_price and current_price < target_price * 0.85:
        score += 10
        reasons.append("Undervalued")
    if peg < 1.5 or pe < 15:
        score += 10
        reasons.append("Good Valuation")
    if peg > 3:
        score -= 10
        reasons.append("Overvalued")

    # Growth
    if eps_growth > 10 or sales_growth > 10:
        score += 10
        reasons.append("High Growth")

    # Risk
    if beta > 1.5:
        score -= 5
        reasons.append("High Beta")
    if short_float > 5:
        score -= 5
        reasons.append("High Short Float")

    return score, ", ".join(reasons)

In [100]:
# ---------- MAIN ANALYSIS (unchanged) ----------
price_data, asset_values, category_values = {}, {}, {}
total_value = 0

In [102]:
for cat in ['stocks', 'crypto']:
    cat_val = 0
    for tk, qty in portfolio[cat].items():
        
        # 👉 Use Alpaca for stocks, Twelve Data for crypto
        if cat == 'stocks':
            df = fetch_stock_history(tk, LOOKBACK_DAYS)  # Alpaca (IEX)
        else:
            df = fetch_crypto_history(tk, LOOKBACK_DAYS)  # Twelve Data

        if df.empty:
            continue
        
        df = add_indicators(df)
        price_data[tk] = df
        
        val = qty * df['close'].iloc[-1]
        asset_values[tk] = val
        cat_val += val

    category_values[cat] = cat_val
    total_value += cat_val

In [104]:
total_value

57927.36080796399

In [106]:
for fixed in ['bonds','currencies']:
    category_values[fixed] = portfolio[fixed]
    total_value += portfolio[fixed]

In [110]:
# Build table
rows = []
for cat in ['stocks','crypto']:
    tgt_pct = target_allocation[cat]
    act_val = category_values[cat]
    act_pct = act_val/total_value*100
    drift = act_pct - tgt_pct

    for tk, qty in portfolio[cat].items():
        df = price_data.get(tk)
        if df is None: continue
        latest = df.iloc[-1]
        val = asset_values[tk]
        alloc_pct = val/total_value*100
        tgt_asset_pct = tgt_pct*(val/act_val)
        technicals = check_technical_signals(latest)
        fundamentals = fundamentals_dict.get(tk, {})  # dict of ticker → fundamentals
        score, reason = intelligent_score(technicals, fundamentals, drift, alloc_pct, tgt_asset_pct)
        action = score_to_action(score)
        rows.append({
            "Category":cat,
            "Ticker":tk,
            "Qty":qty,
            "Curr $":round(val,2),
            "Curr %":round(alloc_pct,2),
            "Tgt %":round(tgt_asset_pct,2),
            "Drift %":round(drift,2),
            "RSI":round(latest['RSI'],2),
            "MACD":round(latest['MACD'],2),
            "Market": "Bull" if latest["MACD"] > latest["MACD_Signal"] else "Bear",
            "Trend":trend(latest['MA20'], latest['MA50']),
            "BB U":round(latest['BB_Upper'],2),
            "BB L":round(latest['BB_Lower'],2),
            "Vol20d":round(latest['Vol_20d'],2),
            "Score":score,
            "Action":action
        })

In [112]:
# ---------- OUTPUT ----------
df = pd.DataFrame(rows)
print("\n📊 Smart Rebalancing Recommendations:\n")
print(df.to_string(index=False))


📊 Smart Rebalancing Recommendations:

Category  Ticker        Qty   Curr $  Curr %  Tgt %  Drift %    RSI    MACD Market   Trend      BB U     BB L  Vol20d  Score         Action
  stocks    AAPL   5.010683   996.42    0.90   0.87     1.37  46.14   -1.32   Bull ↘️ Down    220.77   179.52   10.31     10         ❌ Hold
  stocks    ABBV   0.148790    29.17    0.03   0.03     1.37  68.50    0.13   Bull ↘️ Down    201.08   163.72    9.34     35         ❌ Hold
  stocks    ADBE   0.074650    28.45    0.03   0.02     1.37  76.80   -2.05   Bull ↘️ Down    385.22   331.41   13.45     35         ❌ Hold
  stocks     ALB  19.197652  1112.50    1.00   0.97     1.37  51.57   -2.47   Bull ↘️ Down     62.21    50.90    2.83      0         ❌ Hold
  stocks     AMD   0.206640    20.79    0.02   0.02     1.37  60.89   -0.15   Bull ↘️ Down    104.01    80.46    5.89     40 🔄 Optional Buy
  stocks    AMZN   0.172430    32.13    0.03   0.03     1.37  54.15   -1.17   Bull ↘️ Down    195.97   167.34    7.16    

In [90]:
df

,Category,Ticker,Qty,Curr $,Curr %,Tgt %,Drift %,RSI,MACD,Market,Trend,BB U,BB L,Vol20d,Action
0,stocks,AAPL,5.010683,996.42,0.90,0.87,1.38,46.14,-1.32,Bull,↘️ Down,220.77,179.52,10.31,❌ Wait (No sell trigger)
1,stocks,ABBV,0.148790,29.17,0.03,0.03,1.38,68.50,0.13,Bull,↘️ Down,201.08,163.72,9.34,❌ Wait (No sell trigger)
2,stocks,ADBE,0.074650,28.45,0.03,0.02,1.38,76.80,-2.05,Bull,↘️ Down,385.22,331.41,13.45,⚠️ High risk (volatile or shorted)
3,stocks,ALB,19.197652,1112.50,1.00,0.97,1.38,51.57,-2.47,Bull,↘️ Down,62.21,50.90,2.83,⚠️ Data error
4,stocks,AMD,0.206640,20.79,0.02,0.02,1.38,60.89,-0.15,Bull,↘️ Down,104.01,80.46,5.89,⚠️ High risk (volatile or shorted)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,stocks,WFC,0.119550,8.82,0.01,0.01,1.38,88.77,0.95,Bull,↘️ Down,74.92,59.14,3.95,✅ Sell (Profit-taking in bull)
77,stocks,WMB,0.458750,27.61,0.02,0.02,1.38,62.64,0.53,Bull,↗️ Up,61.46,54.36,1.77,❌ Wait (No sell trigger)
78,stocks,XOM,0.406420,41.98,0.04,0.04,1.38,49.66,-1.15,Bear,↘️ Down,110.57,100.04,2.63,❌ Wait (No sell trigger)
79,crypto,BTC/USD,0.114650,10831.80,9.77,9.01,0.84,71.95,2762.90,Bull,↗️ Up,101049.53,82670.64,4594.72,✅ Sell (Profit-taking in bull)


In [2]:
# Dash app for stock scoring engine using Alpaca technicals
import dash
from dash import Dash, dcc, html, dash_table
import pandas as pd
import time
from finvizfinance.quote import finvizfinance
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta
import pandas_ta as ta
import pytz

# Alpaca credentials (replace with your actual keys)
ALPACA_API_KEY = 'AKLE4VA1CEHBBHXNXCHZ'
ALPACA_SECRET_KEY = 'SQiQpXxbecEhS2p5z9l2olfhA135AuMktWUu12pm'
LOOKBACK_DAYS = 400

client = StockHistoricalDataClient(api_key=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY)

app = Dash(__name__)

# Step 1: Get S&P 500 tickers from Wikipedia
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(sp500_url)
sp500_tickers = sp500_table[0]["Symbol"].tolist()

# Step 2: Add major ETFs manually
etf_tickers = ["SPY", "QQQ", "DIA", "IWM", "VTI", "ARKK", "XLF", "XLK", "XLE", "XLV", "XLY", "XLP"]

# Step 3: Combine ticker list
tickers = list(set(sp500_tickers + etf_tickers))

# Optional: limit number for testing
tickers = tickers[:100]

# tickers = ["AAPL", "MSFT", "GOOG", "NVDA", "TSLA", "JPM"]

def get_technicals_alpaca(ticker):
    end_date = datetime.now(pytz.UTC)
    start_date = end_date - timedelta(days=400)

    request_params = StockBarsRequest(
        symbol_or_symbols=ticker,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date,
        feed='iex'
    )
    bars = client.get_stock_bars(request_params).df

    if bars.empty or ticker not in bars.index:
        return {}

    df = bars.loc[ticker].copy()
    df.ta.macd(close='close', append=True)
    df.ta.rsi(close='close', append=True)

    latest = df.iloc[-1]
    macd_bull = latest['MACD_12_26_9'] > latest['MACDs_12_26_9']
    macd_bear = latest['MACD_12_26_9'] < latest['MACDs_12_26_9']
    rsi = latest['RSI_14']
    overbought = rsi > 70
    oversold = rsi < 30

    return {
        "macd_bull": macd_bull,
        "macd_bear": macd_bear,
        "overbought": overbought,
        "oversold": oversold
    }

def intelligent_score(technicals, fundamentals):
    score = 0
    reasons = []

    if technicals.get("macd_bull"):
        score += 20
        reasons.append("MACD Bullish")
    elif technicals.get("macd_bear"):
        score -= 20
        reasons.append("MACD Bearish")

    if technicals.get("oversold"):
        score += 15
        reasons.append("RSI Oversold")
    elif technicals.get("overbought"):
        score -= 15
        reasons.append("RSI Overbought")

    try:
        recom = float(fundamentals.get("Recom", 0))
        target_price = float(fundamentals.get("Target Price", 0))
        current_price = float(fundamentals.get("Price", 0))
        peg = float(fundamentals.get("PEG", 0))
        pe = float(fundamentals.get("P/E", 0))
        beta = float(fundamentals.get("Beta", 1))
        short_float = float(fundamentals.get("Short Float", "0%" ).replace("%", ""))
        eps_growth = float(fundamentals.get("EPS next Y Percentage", "0%" ).replace("%", ""))
        sales_growth = float(fundamentals.get("Sales Q/Q", "0%" ).replace("%", ""))
    except:
        return 0, "⚠️ Data error"

    if 0 < recom < 2:
        score += 15
        reasons.append("Strong Analyst Rating")
    elif recom > 3:
        score -= 10
        reasons.append("Weak Analyst Rating")

    if target_price and current_price < target_price * 0.85:
        score += 10
        reasons.append("Undervalued")
    if peg < 1.5 or pe < 15:
        score += 10
        reasons.append("Good Valuation")
    if peg > 3:
        score -= 10
        reasons.append("Overvalued")

    if eps_growth > 10 or sales_growth > 10:
        score += 10
        reasons.append("High Growth")

    if beta > 1.5:
        score -= 5
        reasons.append("High Beta")
    if short_float > 5:
        score -= 5
        reasons.append("High Short Float")

    return score, ", ".join(reasons)

def score_to_action(score):
    if score >= 70:
        return "✅ Strong Buy"
    elif 40 <= score < 70:
        return "🔄 Optional Buy"
    elif -39 <= score <= 39:
        return "❌ Hold"
    elif -69 <= score < -40:
        return "🔄 Optional Sell"
    else:
        return "✅ Strong Sell"

data = []
for tk in tickers:
    try:
        stock = finvizfinance(tk)
        fundamentals = stock.ticker_fundament()
        technicals = get_technicals_alpaca(tk)
        score, reason = intelligent_score(technicals, fundamentals)
        action = score_to_action(score)
        data.append({
            "Ticker": tk,
            "Price": fundamentals.get("Price"),
            "Target Price": fundamentals.get("Target Price"),
            "Recom": fundamentals.get("Recom"),
            "PEG": fundamentals.get("PEG"),
            "Score": score,
            "Action": action,
            "Reason": reason
        })
        time.sleep(1)
    except Exception as e:
        data.append({"Ticker": tk, "Score": 0, "Action": "❌ Error", "Reason": str(e)})

df = pd.DataFrame(data)

app.layout = html.Div([
    html.H2("📈 Stock Scoring Dashboard (Alpaca + Finviz)"),
    dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in df.columns],
        data=df.to_dict("records"),
        style_data_conditional=[
            {
                'if': {'column_id': 'Score'},
                'backgroundColor': '#D2F8D2',
                'color': 'black'
            },
            {
                'if': {'filter_query': '{Score} < 0', 'column_id': 'Score'},
                'backgroundColor': '#FFD6D6',
                'color': 'black'
            }
        ],
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
        page_size=10
    )
])

if __name__ == '__main__':
    app.run(debug=True)

In [124]:
result_df.to_csv('result2.csv')